In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
DATA_IN_PATH = './data/'
DATA_OUT_PATH = './output/'
TRAIN_CLEAN_DATA = 'train_clean.csv'
TEST_SIZE = 0.2
RANDOM_SEED = 42

In [4]:
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [5]:
reviews = list(train_data['review'])
y = np.array(train_data['sentiment'])

## CountVectorizer

In [6]:
vectorizer = CountVectorizer(analyzer = 'word', max_features = 5000)
# 모델에 적용하기 위한 특징 추출 방법의 CountVectorizer
# anlayzer = 분석 단위, max_features = 벡터의 최대 길이
train_data_features = vectorizer.fit_transform(reviews)

In [7]:
train_data_features
# 25,000개의 데이터가 각 5,000개의 특징값을 가지는 벡터로 표현

<25000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1975048 stored elements in Compressed Sparse Row format>

## 학습, 검증 데이터

In [9]:
from sklearn.model_selection import train_test_split
train_input, eval_input, train_label, eval_label = train_test_split(train_data_features, y, test_size=TEST_SIZE, random_state=RANDOM_SEED)

## 모델

In [10]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100) 
# 랜덤 포레스트 분류기에  100개 의사 결정 트리를 사용한다. (총 100개의 트리를 만들어서 그 결과를 앙상블)

forest.fit( train_input, train_label )

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [13]:
print("Accuracy: %f" % forest.score(eval_input, eval_label))

Accuracy: 0.841200


## 테스트 데이터

In [14]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_reviews = list(test_data['review'])
ids = list(test_data['id'])

In [15]:
test_data_features = vectorizer.transform(test_reviews)

In [17]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
result = forest.predict(test_data_features)
# 위에서 만든 랜덤 포레스트 분류기를 통해 예측값을 가져온다.

output = pd.DataFrame( data={"id": ids, "sentiment": result} )
# 판다스 데이터 프레임을 통해 데이터를 구성해서 output에 넣는다.

output.to_csv( DATA_OUT_PATH + "Count_Forest_model.csv", index=False, quoting=3 )

### 캐글에서 0.84264점으로 492등 정도의 결과